## Notebook for learning how to run simulation batches

First import required model and its parameters.

In [1]:
from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel, KorakianitisMixedModel_parameters, TEMPLATE_TIME_SETUP_DICT
import numpy as np

ImportError: cannot import name 'CollectionCheckStrategy' from 'typeguard' (/opt/anaconda3/envs/ModularCirc/lib/python3.12/site-packages/typeguard/__init__.py)

Introducing `BatchRunner` a new class which is meant to facilitate running batches of simulations.

In [ ]:
from ModularCirc import BatchRunner

Instantiate `BatchRunner` providing the sampling method used to define the batch.

In [ ]:
br = BatchRunner('LHS', 0) #LHS = Latin HyperCube

Provide the json file detailing the parameter ranges, where these are sampled, or values, where they are constant.

```json
{
    COMPONENT_TYPE_1 : {
        COMPONENT_1 : [
            PARAMETER_1 : [VALUE, [SCALE_RANGE_MIN, SCALE_RANGE_MAX]],
            PARAMETER_2 : [CONSTANT],
            PARAMETER_3 : ...,
        ...
        ]
        COMPONENT_2 : ...
        ...
    }
    COMPONENT_TYPE_2 : ...
    ...
}
```

In [ ]:
# Parameters_01 = Korakianitis Model
br.setup_sampler('parameters_02.json')

### **Sample:**

In [ ]:
# Set number of samples 
br.sample(10)

In [ ]:
# Sampled parameter list
br.samples.columns

# Print sampled data
br.samples

### Rescale the timing parameters and map to actual parameters where appropriate.

In [ ]:
TEMPLATE_TIME_SETUP_DICT

In [ ]:
map_ = {
    'delay' : ['la.delay', 'ra.delay'],
    'td0'   : ['lv.td0',   'rv.td0' ],
    'tr'    : ['lv.tr',    'rv.tr'  ],
    'tpww'  : ['la.tpww',  'ra.tpww'],
}
br.map_sample_timings(
    ref_time=1.,
    map=map_
    )

In [ ]:
br.samples.columns

In [ ]:
br._samples[['lv.td', 'rv.td']] = br._samples[['lv.tr', 'rv.tr']].values + br._samples[['lv.td0', 'rv.td0']].values
br._samples.drop(['lv.td0', 'rv.td0'], axis=1, inplace=True)

In [ ]:
br.samples

use `map_vessel_volumes` to define the initial distribution of blood in the vessels

In [ ]:
br.map_vessel_volume()

In [ ]:
# Quick stats on some parameters
br._samples[['sas.v', 'sat.v', 'svn.v', 'pas.v', 'pat.v', 'pvn.v']].describe().T

Provide the model you want to be run on the parameters.

In [ ]:
br.setup_model(model=KorakianitisMixedModel, po=KorakianitisMixedModel_parameters, time_setup=TEMPLATE_TIME_SETUP_DICT)

Set output path

In [ ]:
import os

In [ ]:
path = os.getcwd()
path

Run simulations

In [ ]:
os.system(f'mkdir -p {path+'/Outputs/Out_02'}')
test = br.run_batch(n_jobs=5, output_path=path+'/Outputs/Out_02')

In [ ]:
# Each row of the sampled parameters (realisations) gives 1000 rows (time_ind) of output for the 31 output parameters.
test

In [ ]:
# Output of first realisation
ind = 0
test[ind].loc[ind]


In [ ]:
# Summary stats of output of first realisation
ind = 0
test[ind].loc[ind].describe()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
t = test[ind].loc[ind]['T'] - test[ind].loc[ind]['T'].loc[0]

fig, ax = plt.subplots()

ax.plot(t, test[ind].loc[ind]['p_pat'], label='lv')


ax.set_xlabel('seconds')
ax.set_ylabel('ml')

ax.legend()
ax.set_title('Volume transients')

plt.show()

In [ ]:
t = test[ind].loc[ind]['T'] - test[ind].loc[ind]['T'].loc[0]

fig, ax = plt.subplots()

ax.plot(t, test[ind].loc[ind]['v_lv'], label='lv')
ax.plot(t, test[ind].loc[ind]['v_la'], label='la')

ax.set_xlabel('seconds')
ax.set_ylabel('ml')

ax.legend()
ax.set_title('Volume transients')

plt.show()

Here is the pressure trace in pulmonary arterial tree:

In [ ]:
p_pat_raw = test[ind].loc[ind]['p_pat']
p_pat_raw

In [ ]:
T     = test[ind].loc[ind]['T'].values.copy()

In [ ]:
T_resample = np.linspace(T[0], T[-1], 100)

If we want to resample we do these steps:

In [ ]:
# Interpolate pressure for 100 timesteps from 1000
p_pat_resampled = np.interp(T_resample, T, p_pat_raw)
p_pat_resampled

Here is the cardiac output:

In [ ]:
q_pat = test[ind].loc[ind]['q_pat'].values.copy()

In [ ]:
q_pat

In [ ]:
CO = np.sum(q_pat) * (T[1] - T[0]) / (T[-1] - T[0]) * 60. / 1000. # L / min
CO